# 준비

In [ ]:
import warnings
warnings.simplefilter('ignore')

# ライブラリのインポート
import pickle as pkl
import pandas as pd
from sklearn.metrics import accuracy_score
from IPython.display import Image
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel
from skater.util.logger import _INFO

In [ ]:
# 데이터, 모델 읽어들임
train = pd.read_csv('train_proc.csv')
with open('lgbm_model.pkl', 'rb') as f:
    model = pkl.load(f)
train_X, train_y = train.drop(columns='Survived'), train['Survived']
train_pred = model.predict(train_X)

# 의사결정 트리 대리 모델 출력

In [ ]:
interpreter = Interpretation(train_X, feature_names=model.feature_name_)
model_inst = InMemoryModel(model.predict, examples=train_X,
                           model_type='classifier', unique_values=[0, 1],
                           feature_names=model.feature_name_, 
                           target_names=['Not Survived','Survived'], 
                           log_level=_INFO)
surrogate_explainer = interpreter.tree_surrogate(oracle=model_inst, seed=42, max_depth=3)
surrogate_explainer.fit(train_X, train_y, use_oracle=True, prune='post',scorer_type='default')

In [ ]:
# 충실도의 계산 (LghtGBM모델의 예측 결과에 대한 Accuracy）
accuracy_score(train_pred, surrogate_explainer.predict(train_X))

In [ ]:
# 의사결정트리 대리 모델의 그래프화
surrogate_explainer.plot_global_decisions(colors=['coral','lightsteelblue','darkkhaki'],
                                          file_name='titanic_skater_tree.png')

In [ ]:
Image('titanic_skater_tree.png')

# scikit-learn의 의사결정트리에 의한 대리모델 생성

In [ ]:
# 라이브러리 임포트
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import pydotplus

In [ ]:
# 의사결정트리에 의한 LightGBM 모델의 예측 결과의 학습
tree = DecisionTreeClassifier(max_depth=3, ccp_alpha=0.0)
tree.fit(train_X, train_pred)
dot_data = export_graphviz(tree, filled=True, rounded=True, feature_names=model.feature_name_, 
                           class_names=['Not Survived','Survived'])

In [ ]:
# 충실도의 계산
tree.score(train_X, train_pred)

In [ ]:
# 의사결정트리의 그래프화
pydotplus.graph_from_dot_data(dot_data).write_png('titanic_sklearn_tree_ccpalpha=0.00(default).png')
Image('titanic_sklearn_tree_ccpalpha=0.00(default).png')

● ccp_alpha=0.01で実行

In [ ]:
# 의사 결정 트리에 의한 LightGBM 모델의 예측 결과 학습
tree = DecisionTreeClassifier(max_depth=3, ccp_alpha=0.01)
tree.fit(train_X, train_pred)
dot_data = export_graphviz(tree, filled=True, rounded=True, feature_names=model.feature_name_, 
                           class_names=['Not Survived','Survived'])

In [ ]:
# 충실도 계산
tree.score(train_X, train_pred)

In [ ]:
# 의사결정트리의 그래프화
pydotplus.graph_from_dot_data(dot_data).write_png('titanic_sklearn_tree_ccpalpha=0.01.png')
Image('titanic_sklearn_tree_ccpalpha=0.01.png')

● ccp_alpha=0.1로 실행

In [ ]:
# 의사결정트리에 의한 LightGBM 모델의 예측 결과 학습
tree = DecisionTreeClassifier(max_depth=3, ccp_alpha=0.1)
tree.fit(train_X, train_pred)
dot_data = export_graphviz(tree, filled=True, rounded=True, feature_names=model.feature_name_, 
                           class_names=['Not Survived','Survived'])

In [ ]:
# 충실도 계산
tree.score(train_X, train_pred)

In [ ]:
# 의사결정 트리 그래프화
pydotplus.graph_from_dot_data(dot_data).write_png('titanic_sklearn_tree_ccpalpha=0.1.png')
Image('titanic_sklearn_tree_ccpalpha=0.1.png')